In [158]:
import numpy as np
import cv2

Now we will load the data which we created in past

In [159]:
with_mask= np.load('with_mask.npy')
without_mask= np.load('without_mask.npy')

In [166]:
with_mask= with_mask.reshape(90,50*50*3) # now as the data is in 3 dimensional so we convert it to 2 dimension
without_mask= without_mask.reshape(90,50*50*3)

In [167]:
X=np.r_[with_mask,without_mask] #now we collect al the data into single variable
X.shape

(180, 7500)

In [168]:
labels= np.zeros(X.shape[0]) # now we label the data as mask and no mask 0 for mask and 1 for no mask

In [171]:
names = {0:'mask',1:'no mask'}

Now we are going to use the svm classifier for the classification purpose

In [172]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [173]:
from sklearn.model_selection import train_test_split

In [174]:
x_train, x_test, y_train, y_test = train_test_split(X,labels,test_size=0.25)#split data for training and testin purpose

In [175]:
from sklearn.decomposition import PCA

In [176]:
pca= PCA(n_components=3) #now we covert the coloums 7500 coloums to 3
x_train=pca.fit_transform(x_train)

In [178]:
x_train, x_test, y_train, y_test = train_test_split(X,labels,test_size=0.30) # it is optional if the overfitting occurs

In [179]:
svm = SVC(kernel='linear')
svm.fit(x_train, y_train)

SVC(kernel='linear')

In [180]:
#x_test= pca.fit_transform(x_test)
y_pred= svm.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred)

now after training and testing we are going to run our actual model

In [188]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cv2.VideoCapture(0)
capture=cv2.VideoCapture(0)
data = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    if flag:
        faces= face_cascade.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),4)
            face = img[y:y+h, x:x+h, :]
            face = cv2.resize(face,(50,50))
            face = face.reshape(1,-1)
            pred= svm.predict(face)
            n = names[int(pred)]
            cv2.putText(img,n,(x,y),font,1,(0,0,255),2)
            #print(n)
            cv2.imshow('result',img)
        if cv2.waitKey(2) == 27:
                break
capture.release
cv2.destroyAllWindows()